# CAR PRICE PREDICTION

In [4]:
# importing libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
import pickle
from mlxtend.plotting import plot_decision_regions

In [5]:
# loading dataset
df = pd.read_csv('audi.csv')

In [6]:
# first 10 values of dataset
df.head(10)

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,A1,2017,12500,Manual,15735,Petrol,150,55.4,1.4
1,A6,2016,16500,Automatic,36203,Diesel,20,64.2,2.0
2,A1,2016,11000,Manual,29946,Petrol,30,55.4,1.4
3,A4,2017,16800,Automatic,25952,Diesel,145,67.3,2.0
4,A3,2019,17300,Manual,1998,Petrol,145,49.6,1.0
5,A1,2016,13900,Automatic,32260,Petrol,30,58.9,1.4
6,A6,2016,13250,Automatic,76788,Diesel,30,61.4,2.0
7,A4,2016,11750,Manual,75185,Diesel,20,70.6,2.0
8,A3,2015,10200,Manual,46112,Petrol,20,60.1,1.4
9,A1,2016,12000,Manual,22451,Petrol,30,55.4,1.4


In [7]:
df.shape

(10668, 9)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10668 entries, 0 to 10667
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         10668 non-null  object 
 1   year          10668 non-null  int64  
 2   price         10668 non-null  int64  
 3   transmission  10668 non-null  object 
 4   mileage       10668 non-null  int64  
 5   fuelType      10668 non-null  object 
 6   tax           10668 non-null  int64  
 7   mpg           10668 non-null  float64
 8   engineSize    10668 non-null  float64
dtypes: float64(2), int64(4), object(3)
memory usage: 750.2+ KB


In [9]:
df.isna().sum()

model           0
year            0
price           0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
dtype: int64

In [10]:
df.describe()

,year,price,mileage,tax,mpg,engineSize
count,10668.000000,10668.000000,10668.000000,10668.000000,10668.000000,10668.000000
mean,2017.100675,22896.685039,24827.244001,126.011436,50.770022,1.930709
std,2.167494,11714.841888,23505.257205,67.170294,12.949782,0.602957
min,1997.000000,1490.000000,1.000000,0.000000,18.900000,0.000000
25%,2016.000000,15130.750000,5968.750000,125.000000,40.900000,1.500000
50%,2017.000000,20200.000000,19000.000000,145.000000,49.600000,2.000000
75%,2019.000000,27990.000000,36464.500000,145.000000,58.900000,2.000000
max,2020.000000,145000.000000,323000.000000,580.000000,188.300000,6.300000


In [11]:
df.duplicated().sum()

103

In [12]:
print("Percentage of Duplicated data: " ,(103/10668)*100)

Percentage of Duplicated data:  0.965504311961005


In [13]:
# We will drop duplicates as the count of duplicate data is less
df.drop_duplicates(inplace=True)

In [14]:
df.duplicated().sum()

0

In [15]:
X = df.iloc[:,[0,1,3,4,5,6,7,8]].values
y = df.iloc[:,[2]].values

In [16]:
X,y

(array([[' A1', 2017, 'Manual', ..., 150, 55.4, 1.4],
        [' A6', 2016, 'Automatic', ..., 20, 64.2, 2.0],
        [' A1', 2016, 'Manual', ..., 30, 55.4, 1.4],
        ...,
        [' A3', 2020, 'Manual', ..., 150, 49.6, 1.0],
        [' Q3', 2017, 'Automatic', ..., 150, 47.9, 1.4],
        [' Q3', 2016, 'Manual', ..., 150, 47.9, 1.4]], dtype=object),
 array([[12500],
        [16500],
        [11000],
        ...,
        [17199],
        [19499],
        [15999]], dtype=int64))

In [17]:
# Encodig Object types and converting to numerical data
le_fuel = LabelEncoder()
le_transmission = LabelEncoder()
le_fuel = LabelEncoder()

In [18]:
X[:,0] = le_fuel.fit_transform(X[:,0])
X[:,-4] = le_fuel.fit_transform(X[:,-4])
X[:,2] = le_transmission.fit_transform(X[:,0])

In [19]:
# feature Scalling
sc = StandardScaler()
X = sc.fit_transform(X)

In [20]:
pd.DataFrame(X)

,0,1,2,3,4,5,6,7
0,-1.119276,-0.039002,-1.119276,-0.393254,1.053589,0.357402,0.351966,-0.884062
1,-0.158819,-0.500425,-0.158819,0.479662,-0.951665,-1.571222,1.030836,0.111173
2,-1.119276,-0.500425,-1.119276,0.212815,1.053589,-1.422867,0.351966,-0.884062
3,-0.543002,-0.039002,-0.543002,0.042479,-0.951665,0.283224,1.269983,0.111173
4,-0.735093,0.883845,-0.735093,-0.979108,1.053589,0.283224,-0.095471,-1.547551
...,...,...,...,...,...,...,...,...
10560,-0.735093,1.345269,-0.735093,-0.892959,1.053589,0.283224,-0.095471,-1.547551
10561,-0.735093,1.345269,-0.735093,-0.979961,1.053589,0.357402,-0.095471,-1.547551
10562,-0.735093,1.345269,-0.735093,-1.038346,1.053589,0.357402,-0.095471,-1.547551
10563,0.609547,-0.039002,0.609547,-0.695585,1.053589,0.357402,-0.226616,-0.884062


In [21]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [22]:
n_estimators = [int(x) for x in np.linspace(start = 80, stop = 1500, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(6, 45, num = 5)]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10]

random_grid={'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf}

In [23]:
RF = RandomForestRegressor()

In [24]:
RCV = RandomizedSearchCV(estimator=RF,param_distributions=random_grid,scoring='neg_mean_squared_error',n_iter=3,cv=3,random_state=42, n_jobs = -1)

In [25]:
RCV.fit(X_train,y_train)

c:\Users\Divyansh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:542: FitFailedWarning: 
3 fits failed out of a total of 9.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Divyansh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 890, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Divyansh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1344, in wrapper
    estimator._validate_params()
  File "c:\Users\Divyansh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklear

RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=3, n_jobs=-1,
                   param_distributions={'max_depth': [6, 15, 25, 35, 45],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 5, 10],
                                        'min_samples_split': [2, 5, 10, 15,
                                                              100],
                                        'n_estimators': [80, 237, 395, 553, 711,
                                                         868, 1026, 1184, 1342,
                                                         1500]},
                   random_state=42, scoring='neg_mean_squared_error')

In [26]:
y_pred = RCV.predict(X_test)

In [27]:
y_pred

array([11747.71574052, 48229.36845117, 21659.36061619, ...,
       33445.63897409, 33668.69541774, 22428.94035781])

In [28]:
print("Mean Absolute Error:",mean_absolute_error(y_test,y_pred))

Mean Absolute Error: 1604.7446031250288


In [29]:
print("Mean Squared Error",mean_squared_error(y_test,y_pred))

Mean Squared Error 6674642.110360708


In [30]:
print("R2 Score:",r2_score(y_test,y_pred))

R2 Score: 0.9528391537036052


In [31]:
# Storing Model for future use
pickle.dump(RCV,open('model.pkl','wb'))